In [7]:
# Install Packages
!pip install google-api-python-client google-cloud-bigquery-storage pycountry

In [ ]:
# --- 1. SETUP AND AUTHENTICATION ---

from googleapiclient.discovery import build
from google.cloud import bigquery
from datetime import datetime
import pandas as pd
import pycountry
import time

# Initialize the YouTube Data API client
youtube = build("youtube", "v3", developerKey=os.getenv("YOUTUBE_API_KEY"))

print("✅ YouTube client initialized successfully")

✅ YouTube client initialized successfully


In [9]:
# --- 2. DEFINE REQUEST PARAMETERS ---

# --- Define African regions with all 54 countries ---
regions = {
    "East Africa": ["KE", "UG", "TZ", "RW", "BI", "ET", "SO", "DJ", "ER", "SC", "MG", "MU"],
    "West Africa": ["NG", "GH", "CI", "SN", "TG", "SL", "LR", "GM", "BF", "BJ", "NE", "ML", "GN", "GW", "CV"],
    "North Africa": ["EG", "DZ", "MA", "TN", "LY", "SD"],
    "Southern Africa": ["ZA", "NA", "BW", "MZ", "ZW", "ZM", "LS", "SZ", "AO", "MW"],
    "Central Africa": ["CM", "CD", "CG", "GA", "GQ", "CF", "TD", "ST"]
}

# The API returns results in pages; this keeps track of which page to fetch next
next_page_token = None

In [10]:
# --- Initialize ---
videos = []

for region, countries in regions.items():
    print(f"Collecting data for {region} ✅")
    
    for country in countries:
        # --- Get full country name ---
        try:
            country_name = pycountry.countries.get(alpha_2=country).name
        except:
            country_name = "Unknown"
        
        try:
            next_page_token = None
            while True:
                request = youtube.videos().list(
                    part="snippet,statistics",
                    chart="mostPopular",
                    regionCode=country,
                    maxResults=50,
                    pageToken=next_page_token
                )
            
                response = request.execute()

                # --- Create a category map ---
                category_response = youtube.videoCategories().list(
                    part="snippet",
                    regionCode=country
                ).execute()
                
                category_map = {item["id"]: item["snippet"]["title"] for item in category_response.get("items", [])}

                for item in response.get("items", []):
                    snippet = item.get("snippet", {})
                    stats = item.get("statistics", {})
            
                    # Get category info
                    category_id = snippet.get("categoryId")
                    category_name = category_map.get(category_id, "Unknown")

                    videos.append({
                        "country_code": country,
                        "country_name": country_name,  # ✅ added country name
                        "region": region,
                        "video_id": item.get("id"),
                        "title": snippet.get("title"),
                        "description": snippet.get("description"),
                        "channel_title": snippet.get("channelTitle"),
                        "category_id": category_id,
                        "category_name": category_name,  # ✅ new field
                        "published_at": snippet.get("publishedAt"),
                        "tags": snippet.get("tags", []),
                        "view_count": int(stats.get("viewCount", 0)),
                        "like_count": int(stats.get("likeCount", 0)),
                        "comment_count": int(stats.get("commentCount", 0))
                    })

                next_page_token = response.get("nextPageToken")
                if not next_page_token:
                    break

        except Exception as e:
            # print(f"  ⚠️ Skipping {country} — Not supported or error occurred: {e}")
            continue

print(f"Total videos retrieved across Africa: {len(videos)} ✅")

Total videos retrieved across Africa: 2109 ✅


In [11]:
# Convert to DataFrame
bigdata = pd.DataFrame(videos)
bigdata.drop_duplicates(subset=["video_id", "country_code", "country_name", "region"], keep="first", inplace=True)
bigdata = bigdata.reset_index(drop=True)

In [12]:
bigdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2109 entries, 0 to 2108
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   country_code   2109 non-null   object
 1   country_name   2109 non-null   object
 2   region         2109 non-null   object
 3   video_id       2109 non-null   object
 4   title          2109 non-null   object
 5   description    2109 non-null   object
 6   channel_title  2109 non-null   object
 7   category_id    2109 non-null   object
 8   category_name  2109 non-null   object
 9   published_at   2109 non-null   object
 10  tags           2109 non-null   object
 11  view_count     2109 non-null   int64 
 12  like_count     2109 non-null   int64 
 13  comment_count  2109 non-null   int64 
dtypes: int64(3), object(11)
memory usage: 230.8+ KB


In [13]:
bigdata.head()

,country_code,country_name,region,video_id,title,description,channel_title,category_id,category_name,published_at,tags,view_count,like_count,comment_count
0,KE,Kenya,East Africa,ReDAxNBuZLc,D Voice feat Zuchu - Baridi (Official Visualizer),https://www.youtube.com/channel/UCxKoKHs7YwjE_...,D Voice,10,Music,2026-01-17T21:02:26Z,"[music, diamond platnumz, swahili kid, umenifu...",629524,16682,1011
1,KE,Kenya,East Africa,az-Yrg-nDEU,DEM WA SKUMA - One Woman.Two Lives ( Official ...,A fearless sukuma seller living a double life ...,KADUSKO Tv,23,Comedy,2026-01-20T11:00:05Z,"[KADUSKO Tv, KADUSKO, Comedy]",35999,318,19
2,KE,Kenya,East Africa,1A5SFR3bPfs,Senegal v Egypt | Match in 3 | CAF Africa Cup ...,Highlights from the match between Senegal and ...,SuperSport,17,Sports,2026-01-14T19:11:18Z,"[Egypt, Senegal, Senegal vs. Egypt]",494410,3665,280
3,KE,Kenya,East Africa,SOJpE1KMUbo,Dave - Raindance (ft. Tems),Get tickets to 'The Boy Who Played the Harp' t...,Santan Dave,10,Music,2026-01-09T18:27:49Z,"[dave, santan dave, santandave, tems, dave x t...",15032194,355499,9453
4,KE,Kenya,East Africa,1F6BCHAfGDc,The Night Agent: Season 3 | Official Trailer |...,"What began with a call, became a calling. The ...",Netflix,24,Entertainment,2026-01-21T14:00:00Z,"[Action, Action Series, Brittany Snow, Chelsea...",229602,6397,350


In [14]:
bigdata.tail()

,country_code,country_name,region,video_id,title,description,channel_title,category_id,category_name,published_at,tags,view_count,like_count,comment_count
2104,ZW,Zimbabwe,Southern Africa,Bx2ejavSVSw,Minecraft Speedrunner VS 3 Hunters REVIVAL,You asked for it. The path to 6 hunters contin...,Dream,20,Gaming,2025-12-24T08:40:10Z,"[minecraft, dream, dream minecraft]",8957566,304413,16578
2105,ZW,Zimbabwe,Southern Africa,w1JvpE2XiSo,"Tyler ICU, XDuppy, AL Xapo, Benzoo, Optimist M...",,Tyler ICU,10,Music,2025-12-24T22:00:07Z,[],978993,19329,493
2106,ZW,Zimbabwe,Southern Africa,ODhUyM9dxu8,"Extreme Hide & Seek In Rivals, But I Used GLIT...",Join My Roblox Group!\nhttps://www.roblox.com/...,More Techy,20,Gaming,2025-12-21T17:00:41Z,"[roblox, funnyroblox, techyblox, techy, crazy ...",1099585,11000,575
2107,ZW,Zimbabwe,Southern Africa,hbhAfcua6O4,Vybz Kartel - Victorious (Official Music Video),Vybz Kartel - Victorious (Official Music Video...,VybzKartelVEVO,10,Music,2025-12-20T17:01:13Z,"[Vybz Kartel, popcaan, onerpm, dancehall, danc...",1151203,34129,3102
2108,ZW,Zimbabwe,Southern Africa,1GiEjt2D5Fc,How i got Arrested...,I Played EVERY Fortnite Rip-Off\n𝐒𝐔𝐁𝐒𝐂𝐑𝐈𝐁𝐄 👉 ​...,Royalty Gaming,20,Gaming,2025-12-20T20:07:41Z,"[royalty gaming, royalty family gaming, ferran...",3503896,32864,4463


In [15]:
bigdata['category_name'].value_counts()

category_name
Gaming              993
Music               702
People & Blogs      153
Entertainment       150
Sports               65
Film & Animation     26
Education             5
Unknown               5
Howto & Style         5
Comedy                4
Pets & Animals        1
Name: count, dtype: int64

In [18]:
# Initialize BigQuery client
client = bigquery.Client(project='data-storage-485106')

In [16]:
bigdata['tags'] = bigdata['tags'].astype(str)

In [19]:
# Define Table ID
table_id = 'data-storage-485106.youtube.trending_now'

# Export Data to BigQuery
job = client.load_table_from_dataframe(bigdata, table_id)
while job.state != 'DONE':
    time.sleep(2)
    job.reload()
    print(job.state)

DONE


In [20]:
# Define SQL Query to Retrieve Open Weather Data from Google Cloud BigQuery
sql = (
    'SELECT *'
    'FROM `data-storage-485106.youtube.trending_now`'
    'ORDER BY country_code'
      )
    
# Run SQL Query
data = client.query(sql).to_dataframe()

In [21]:
# Delete Original Table
client.delete_table(table_id)
print(f"Table deleted successfully.")
    
# Remove Duplicate Records
data.drop_duplicates(subset=["video_id", "country_code", "country_name", "region"], inplace=True)

Table deleted successfully.


In [22]:
data['tags'] = data['tags'].astype(str)

In [24]:
# Define the dataset ID and table ID
dataset_id = 'youtube'
table_id = 'trending_now'
    
# Define the BigQuery schema for YouTube trending videos
schema = [
    bigquery.SchemaField("country_code", "STRING"),
    bigquery.SchemaField("country_name", "STRING"),
    bigquery.SchemaField("region", "STRING"),
    bigquery.SchemaField("video_id", "STRING"),
    bigquery.SchemaField("title", "STRING"),
    bigquery.SchemaField("description", "STRING"),
    bigquery.SchemaField("channel_title", "STRING"),
    bigquery.SchemaField("category_id", "STRING"),
    bigquery.SchemaField("category_name", "STRING"),
    bigquery.SchemaField("published_at", "STRING"),
    bigquery.SchemaField("tags", "STRING"),  # optional: convert list to string before upload
    bigquery.SchemaField("view_count", "INTEGER"),
    bigquery.SchemaField("like_count", "INTEGER"),
    bigquery.SchemaField("comment_count", "INTEGER")
]

# Define the table reference
table_ref = client.dataset(dataset_id).table(table_id)
    
# Create the table object
table = bigquery.Table(table_ref, schema=schema)

try:
    # Create the table in BigQuery
    table = client.create_table(table)
    print(f"Table {table.table_id} created successfully.")
except Exception as e:
    print(f"Table {table.table_id} failed")

Table trending_now created successfully.


In [25]:
# Define the BigQuery table ID
table_id = 'data-storage-485106.youtube.trending_now'

# Load the data into the BigQuery table
job = client.load_table_from_dataframe(data, table_id)

# Wait for the job to complete
while job.state != 'DONE':
    time.sleep(2)
    job.reload()
    print(job.state)

DONE


In [ ]:
# Define SQL Query to Retrieve Open Weather Data from Google Cloud BigQuery
sql = (
    'SELECT *'
    'FROM `data-storage-485106.youtube.trending_now`'
    'ORDER BY country_name'
      )
    
# Run SQL Query
data = client.query(sql).to_dataframe()

In [ ]:
data.shape

,country_code,country_name,region,video_id,title,description,channel_title,category_id,category_name,published_at,tags,view_count,like_count,comment_count
0,DZ,Algeria,North Africa,sr8fIUDD-vg,POSSO Fazer MEGA ROBÔS no Minecraft!,📸 Meu Instagram:\nhttps://www.instagram.com/_a...,Athos,20,Gaming,2026-01-21T13:31:06Z,"['minecraft', 'athos', 'athos gamer', 'minecra...",579800,5651,123
1,DZ,Algeria,North Africa,_9e0kuQlNhs,English SAKURA SCHOOL SIMULATOR: 😍 Excited str...,Hey folks! Watch me play Sakura school simulat...,ahiplayz,20,Gaming,2026-01-21T11:07:07Z,[],416165,1338,0
2,DZ,Algeria,North Africa,DBC2KsLkoV0,Golo l oumi semhi liya قولو لمي سحي ليا (Speci...,Provided to YouTube by DistroKid\n\nGolo l oum...,kamikabro - Topic,10,Music,2026-01-16T09:37:14Z,"['kamikabro', 'hajar egypte alam el fan mazzik...",64133,1064,45
3,DZ,Algeria,North Africa,7dgNijJQO_M,Long Slide Game With Cow Elephant Gorilla Hipp...,#animals3dbin #longslidegame #funny3danimals ...,Flygame Animals,1,Film & Animation,2026-01-21T05:05:31Z,[],750882,11190,198
4,DZ,Algeria,North Africa,epzngHfK_xE,Choo Choo Charles is Hunting me | Horror Gameplay,"Hello People , In this Video we are playing th...",Chahat Gaming 3.0,20,Gaming,2026-01-21T21:49:05Z,[],167800,710,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2104,ZW,Zimbabwe,Southern Africa,wby65kBRWzk,STEAL A BRAINROT GIVEAWAY LIVE | STEAL A BRAIN...,STEAL A BRAINROT GIVEAWAY LIVE | STEAL A BRAIN...,Joxy,20,Gaming,2026-01-17T18:01:04Z,"['steal a brainrot giveaway', 'steal a brainro...",874034,10079,49
2105,ZW,Zimbabwe,Southern Africa,lCqLnjLm6jE,🔴SENEGAL vs MOROCCO - Africa Cup of Nations 20...,Subscribe Please! \nPes 21 - SENEGAL vs MOROCC...,Banchik,20,Gaming,2026-01-18T23:08:22Z,"['pes 2021', 'pes 2021 gameplay', 'soccer', 'p...",414545,570,2
2106,ZW,Zimbabwe,Southern Africa,ErnWUbjOKoU,SIDEMEN AMONG US DUMPER ROLE: COLLECT DEAD BOD...,🍗: Order food NOW at: https://www.eatsides.com...,MoreSidemen,22,People & Blogs,2025-12-23T19:20:00Z,"['sidemen', 'moresidemen', 'miniminter', 'ksi'...",5172071,149499,4624
2107,ZW,Zimbabwe,Southern Africa,DjNKhpOC6Ag,Shone - Kuchema (Official Music Video),Video was shot in Kuwadzana Extension where Sh...,Shone,10,Music,2026-01-16T08:00:06Z,[],15177,670,69
